## 1. Load voxel model

In [ ]:
from setup.voxel_setup import setup_voxel_scene
from common.plot import Plotter
from simulation.simulator import get_irrad_loc_dir, compute_ior_gradient

import taichi as ti
from scipy import ndimage
import numpy as np

%load_ext autoreload
%autoreload 2
# May comment it because the compatibility of this extension is not good
%matplotlib widget 

# debug=True to check boundary access
ti.init(arch=ti.gpu)

SCENE_CFG = {
    # Optional Names: "geometry", "bunny", "footed_glass", "stemmed_glass"
    "Name": "bunny", 
     
    # "HDR Res": (4000, 2000), 
    # "HDR Name": "Dark_night_4k.hdr",

    "HDR Res": (2000, 1000),
    "HDR Name": "Light_wooden_frame_room_2k.hdr",
    
    "NUM XYZ": (128, 128, 128),
    'Floor Ratio': -0.95,

    "Sampler Num": 5,

    "Load Save": True,
}

PROC_CFG = {
    "Gauss Sigma": 4.0,
    "Gauss Radius": 2,

    "Grad Threshold": 0.0,
}

plotter = Plotter(SCENE_CFG)
scene = setup_voxel_scene(SCENE_CFG)

scene.apply_filter(PROC_CFG)
scene.gradient = compute_ior_gradient(scene.ior)
plotter.plot_wavefront(scene.ior, None, None)

## 2. Perform light simulation

In [ ]:
def get_save_name(attribute: str, config: dict) -> str:
    return f"{attribute}_{config['Name']}_{config['NUM XYZ'][0]}_{config['NUM XYZ'][1]}_{config['NUM XYZ'][2]}"

def check_save_exists(savename: str) -> bool:
    
    return False

In [ ]:
scene.irradiance, scene.local_diretion = get_irrad_loc_dir(scene, SCENE_CFG, plotter=plotter,
                                                           num_show_images=3)


In [ ]:
# plotter.plot_irradiance_grid(scene.irradiance)
# plotter.plot_irradiance_slices(scene.irradiance, num_slices=4, z_start=30, z_end=100)
# plotter.plot_local_direction_grid_slices(scene.local_diretion[:, floor_height:, :], num_slices=4, z_start=30, z_end=100)

## 3. Ray marching render

In [ ]:
# res = scene.offline_render([np.array([3.0, 3.0, 0.0])])
# import matplotlib.pyplot as plt
# plt.clf()
# plt.imshow(res[0])
# plt.show()

In [ ]:
scene.rt_render(True)

## 4. Different data structures to store or fit irradiance

In [ ]:
from data.siren import SirenFitter, siren_post_process
from data.mlp import MLP, mlp_post_process
from data.octree import Octree, octree_post_process

In [ ]:
octree = Octree(threshold=14)
octree.construct(scene.irradiance)
print(f"Number of nodes: {len(octree)}")
print(f"Octree Memory usage: {octree.__sizeof__()} bytes")
print(f"In comparison, NumPy Storage Usage: {scene.irradiance.nbytes} bytes")
octree.visualize(plotter, num_slices=4, z_start=30, z_end=100)

In [ ]:

octree_res = octree.init_empty_grid()
octree.fill_grid(octree.root, octree_res, 0, 0, 0, octree.grid_size)
plotter.plot_irradiance_slices(octree_res, threshold=3, num_slices=4, z_start=30, z_end=100)
corrected_octree_res = octree_post_process(octree_res)
plotter.plot_irradiance_slices(corrected_octree_res, threshold=3, num_slices=4, z_start=30, z_end=100)

In [ ]:
assert corrected_octree_res.shape == scene.irradiance.shape, "The shape of the corrected octree result should be the same as the original irradiance grid"
scene.irradiance = corrected_octree_res
scene.rt_render()

In [ ]:

x, y, z = 60, 60, 60
value = octree.query(x, y, z)
value

### 4.1 SIREN to fit irradiance

In [ ]:
siren_fitter= SirenFitter(scene.irradiance, floor_height, sampler_multiplier,
                     hidden_features=256, hidden_layers=3, omega=24)
siren_fitter.fit(total_epochs=24, batch_size=20000, lr=5e-4)

In [ ]:
siren_res = siren_fitter.infer()
plotter.plot_irradiance_slices(siren_res, threshold=3, num_slices=4, z_start=30, z_end=100)
siren_res.shape

In [ ]:
corrected_siren_res = siren_post_process(siren_res, gamma=0.2)
plotter.plot_irradiance_slices(corrected_siren_res, threshold=3, num_slices=4, z_start=30, z_end=100)

In [ ]:
assert corrected_siren_res.shape == scene.irradiance.shape, "The shape of the corrected siren result should be the same as the original irradiance grid"
scene.irradiance = corrected_siren_res
scene.rt_render()

### 4.2 MLP to fit irradiance

In [ ]:
mlp = MLP(scene.irradiance, floor_height, NUM_XYZ, sampler_multiplier, num_epoches=150)

In [ ]:
# Visualize the predicted irradiance field
mlp_predicted_irradiance = mlp.predict(pad=True)
plotter.plot_irradiance_slices(mlp_predicted_irradiance, threshold=3, 
                               num_slices=4, z_start=30, z_end=100)

In [ ]:
corrected_mlp_res = mlp_post_process(mlp_predicted_irradiance, None)
plotter.plot_irradiance_slices(corrected_mlp_res, threshold=3, num_slices=4, z_start=30, z_end=100)

In [ ]:
assert corrected_mlp_res.shape == scene.irradiance.shape, "The shape of the corrected mlp result should be the same as the original irradiance grid"
scene.irradiance = corrected_mlp_res
scene.rt_render()

### 4.3 Octree to store irradiance